<a href="https://colab.research.google.com/github/gschivley/FERC_714/blob/master/FERC714_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FERC 714 hourly demand data

This notebook extracts a couple years of hourly demand data from FERC 714 and starts exploring ways to match the FERC respondents to EIA utility/balancing authority entities in EIA-861. My goal is to match the FERC respondents with IPM regions. It's a working document that I'm sharing with the hope that other people will be able to check and improve on what I've done. If you have questions or want to suggest changes/additional data you can [open an issue](https://github.com/gschivley/FERC_714/issues), find on on twitter ([@gschivley](https://twitter.com/gschivley)), or email me at *greg at carbonimpact dot co*.

In [ ]:
!wget https://raw.githubusercontent.com/gschivley/EIA_Cleaned_Hourly_Electricity_Demand_Code/master/anomaly_screening.py

In [ ]:
from itertools import combinations, chain
import pandas as pd
import numpy as np
from pathlib import Path
import zipfile
import urllib
from joblib import Parallel, delayed

from anomaly_screening import screen_timeseries, make_anomaly_summary

cwd = Path.cwd()
pd.set_option("display.max_columns", 100)

In [ ]:
# Download the FERC 714 data to a temp folder that google is nice enough to host.
url = 'https://www.ferc.gov/docs-filing/forms/form-714/data/form714-database.zip'
save_folder = cwd / "FERC"
save_folder.mkdir(parents=True, exist_ok=True)
urllib.request.urlretrieve(url, save_folder / 'form714-database.zip')
### Unzip it
data_path = save_folder / "form714-database"
with zipfile.ZipFile(save_folder / 'form714-database.zip', 'r') as zfile:
    zfile.extractall(data_path)

In [ ]:
df = pd.read_csv(
    data_path / "Part 3 Schedule 2 - Planning Area Hourly Demand.csv",
    parse_dates=["plan_date"], infer_datetime_format=True
)
df.head()

In [ ]:
respondents = df.loc[df.report_yr >=2011,"respondent_id"].unique()

respondents

In [ ]:
def hour_25_looks_real(df):
    """
    The FERC demand data has a column hour25 for daylight savings. Determine if
    the value there looks like it could be a continuation of the series.
    Sometimes it seems to be a sum of all values for the day or something else
    weird.
    """
    df["hour_25_valid"] = False
    df["hour24_25_ratio"] = df["hour24"] / df["hour25"]
    df.loc[
        (df["hour24_25_ratio"] > 0.6)
        & (df["hour24_25_ratio"] < 1.5),
        "hour_25_valid"
    ] = True
    
    df.loc[df["hour_25_valid"] == False, "hour25"] = np.nan
    
    return df

In [ ]:
# Only keeping 2011/2012 for now. Explore more of the data if you want!
corrected_df = hour_25_looks_real(
    df.loc[(df.report_yr.isin([2011, 2012])), :].copy()
)

In [ ]:
# Check to see how many valid hour25 values there are for each respondent.
# Looks like 311 uses hour25 all the time...
hour_25_count = {}
for r in corrected_df.dropna()["respondent_id"].unique():
    hour_25_count[r] = len(corrected_df.query("respondent_id==@r").dropna())

hour_25_count

In [ ]:
# hour25 values do appear to be on dayslight savings change
corrected_df.query("hour25 > 0 & respondent_id != 311").sort_values("report_yr")

In [ ]:
# Same for a 0 value in hour02, which happens sometimes instead of skipping the hour
corrected_df.query("hour02 == 0 & hour01 != 0").sort_values("report_yr")

In [ ]:
def timezone_to_tz(timezone):
    return 'Etc/GMT{:+}'.format(-timezone)

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [ ]:
hourcols = ['hour{:02.0f}'.format(i) for i in range(1,26)]

# These are my best guesses for all of the timezone values that BAs listed
tz_offset = {
    '1  ': -5,
    "   ": -7,
    "AKS": -9,
    "CST": -6,
    "CPT": -6,
    "MST": -7,
    "PST": -8,
    "PDT": -8,
    "mst": -7,
    "EST": -5,
    "EDT": -5,
    " CS": -6,
    "HST": -10,
    "Est": -5,
    "PPT": -8,
    "MPT": -7,
    "EPT": -5,  
    "MPP": -7,
}
tz_ba = {key: timezone_to_tz(offset) for key, offset in tz_offset.items()}
error_dfs = {}
good_dfs = {}
years = [2011, 2012]
year_hours = {
    2011: 8760,
    2012: 8784
}

for r in respondents:
    # Not all respondents have data for all years
    r_all_years = corrected_df.loc[
            (corrected_df.respondent_id == r) & (corrected_df.report_yr.isin(years)),
            :
        ]
    # Only proceed if there is positive demand over all years (skip if all 0)
    if r_all_years[hourcols].sum().sum() > 0:
        valid_years = sorted(r_all_years["report_yr"].unique().tolist())
        tz = r_all_years["timezone"].values[0]
        dt = pd.date_range(
            f"{valid_years[0]}-01-01", 
            f"{valid_years[-1] + 1}-01-01",
            freq="H",
            closed="left",
            tz=tz_ba[tz]
        )

        df_list = []
        for year in valid_years:
            r_single_year = r_all_years.loc[r_all_years.report_yr == year, :]

            # Try to drop March DST changeover values if there are more valid hours
            # (not nan) than hours in the year.
            # I found that hour02, hour03, and hour24 all had 0 values for at
            # least one respondent.
            # Set the 0 values to np.nan so they can be dropped after melting
            if r_single_year[hourcols].count().sum() > year_hours[year]:
                r_single_year.loc[
                    (r_single_year["hour02"] == 0)
                    & (r_single_year["hour01"] != 0)
                    & (r_single_year["plan_date"].dt.month == 3),
                    "hour02"
                ] = np.nan
                r_single_year.loc[
                    (r_single_year["hour03"] == 0)
                    & (r_single_year["hour01"] != 0)
                    & (r_single_year["plan_date"].dt.month == 3),
                    "hour03"
                ] = np.nan
                r_single_year.loc[
                    (r_single_year["hour24"] == 0)
                    & (r_single_year["hour01"] != 0)
                    & (r_single_year["plan_date"].dt.month == 3),
                    "hour24"
                ] = np.nan
            
            tidy_df = pd.melt(r_single_year, id_vars='plan_date', value_vars=hourcols, 
                 var_name='hour', value_name='demand_MW')
            tidy_df = tidy_df.sort_values(["plan_date", "hour"])

            tidy_df = tidy_df.dropna()
            tidy_df["hour"] = tidy_df["hour"].str[-2:].astype(int)
            tidy_df["respondent_id"] = r

            df_list.append(tidy_df)

        # Concat the years together
        r_df = pd.concat(df_list)
        r_df = r_df.reset_index(drop=True)

        # If the length of one year, all years, or some combination of years
        if len(r_df) in [sum(x) for x in powerset(year_hours.values())]:

            r_df["date_time"] = dt
            columns = ["date_time", "demand_MW", "respondent_id"]
            r_df = r_df.loc[:, columns]
            good_dfs[r] = r_df

        else:
            print(r, len(r_df))
            error_dfs[r] = r_df


From the code above it looks like ~120 BAs have data clean enough that I'm able to extract the correct number of hours. 

\#225 has 2 days with demand in hour25 (daylight savings) but only one of the years has an hour in March with 0 demand. I suppose I could just remove an hour from that day?

\# 311 has demand in hour25 every day (or just about). Interestingly, hour01 and hour02 seem to always have the same values. Maybe just drop hour01 and shift everything over?

## Check the demand data for anomalies

The anomaly checking functions and parameter values below are all from [a notebook](https://github.com/truggles/EIA_Cleaned_Hourly_Electricity_Demand_Code) by Tyler Ruggles, which he developed to screen [hourly demand data from EIA-931](https://www.eia.gov/realtime_grid/#/status?end=20200325T07). I've modified some functions to speed them up. EIA's hourly data only goes back to mid-2015, which is why I'm using FERC 714. Fortunately it looks like the FERC data — once properly extracted — doesn't have many of the anomalies that Tyler found in the EIA data. Tyler also [developed a method](https://github.com/truggles/EIA_Cleaned_Hourly_Electricity_Demand_Code/blob/master/MICE_step.Rmd) to impute missing or anomalous data but at first glance it looks like the FERC demand data can largely be used as-is without too many issues. Let me know if you disagree!

In [ ]:
short_hour_window = 24 # 48 hour moving median (M_{t,48hr})
iqr_hours = 24*5 # width in hours of IQR values of relative deviations from diurnal cycle template (IQR_{dem,t})
nDays = 10 # Used for normalized hourly demand template (h_{t,diurnal}) and 480 hour moving median (M_{t,480hr})
global_dem_cut = 10 # threshold selection for global demand filter
local_dem_cut_up = 3.5 # upwards threshold for local demand filter
local_dem_cut_down = 2.5 # downwards threshold for local demand filter
delta_multiplier = 2 # selection threshold for double-sided delta filter
delta_single_multiplier = 5 # selection threshold for single-sided delta filter
rel_multiplier = 15 # other selection threshold for single-sided delta filter
anomalous_regions_width = 24 # width in hours of anomalous region filter
anomalous_pct = .85 # required pct of good data in anomalous region filter

In [ ]:
name_df_list = Parallel(n_jobs=-1, verbose=10)(delayed(screen_timeseries)(
    name=name, 
    df=df, 
    short_hour_window=short_hour_window,
    iqr_hours=iqr_hours,
    nDays=nDays,
    global_dem_cut=global_dem_cut,
    local_dem_cut_up=local_dem_cut_up,
    local_dem_cut_down=local_dem_cut_down,
    delta_multiplier=delta_multiplier,
    delta_single_multiplier=delta_single_multiplier,
    rel_multiplier=rel_multiplier,
    anomalous_regions_width=rel_multiplier,
    anomalous_pct=anomalous_pct
    ) for name, df in good_dfs.items()
)

In [ ]:
anomaly_dict = {name: df for name, df in name_df_list}
summary_df = make_anomaly_summary(anomaly_dict)
summary_df

In [ ]:
summary_df.describe()

## Matching FERC respondents to EIA-816 utilities

The SI of [Auffhammer et al](https://www.pnas.org/content/pnas/suppl/2017/02/01/1613193114.DCSupplemental/pnas.1613193114.sapp.pdf) describes how they derived geographic coverage of FERC respondents using EIA-861 and the crosswalk between FERC respondents and EIA utilities/BAs. I'm including a little code to start exploring that here.

This is very exploratory! I'm trying to match codes between FERC and 861 but haven't figured out yet if they should be matched to EIA BAs or Utilities. At a minimum it looks like there are 84 FERC respondents that don't have a match in either category of the 2012 861.

A few extra resources that might be helpful:
- SPP [historical hourly load](https://marketplace.spp.org/pages/hourly-load#) back through 2011. The company acronyms (at least for 2011/2012 data) are described in [this document](https://www.nerc.com/pa/rrm/Resources/Monitoring_and_Situational_Awareness_Conference2/2%20Testing%20Your%20Sensitivity%20To%20Loss%20of%20Data_T%20Miller.pdf).
- MISO has [archivled historical hourly load](https://www.misoenergy.org/markets-and-operations/real-time--market-data/market-reports/market-report-archives/#nt=%2FMarketReportType%3ASummary%2FMarketReportName%3AArchived%20Historical%20Regional%20Forecast%20and%20Actual%20Load%20%20(zip)&t=10&p=0&s=MarketReportPublished&sd=desc) for 2007-2014 and [historical hourly load](https://www.misoenergy.org/markets-and-operations/real-time--market-data/market-reports/#nt=%2FMarketReportType%3ASummary%2FMarketReportName%3AHistorical%20Regional%20Forecast%20and%20Actual%20Load%20(xls)&t=10&p=0&s=MarketReportPublished&sd=desc) back through 2013 for Central, East, and West regions.
- ERCOT has [historical hourly load](http://www.ercot.com/gridinfo/load/load_hist/) in each of the 8 weather zones. (NOTE: the IPM region ERC_PHDL has no demand, so all weather zones are split into ERC_WEST and ERC_REST)
- PJM requires a login and API key to access their [historical metered data](https://dataminer2.pjm.com/feed/hrl_load_metered/definition) through dataminer (back to 1993).
- NYISO has hourly load by zone (which look like they group well into IPM regions) back through 2001. I need to figure out the difference between [real time actual load](http://mis.nyiso.com/public/P-58Blist.htm) and [integrated real time actual load](http://mis.nyiso.com/public/P-58Clist.htm).
- ISONE has [hourly load by zone](https://www.iso-ne.com/isoexpress/web/reports/load-and-demand/-/tree/zone-info) back through 2011. Unfortunately, from a quick glance at 2011 it looks like while there are only 8760 hourly demand values the 2am on March 13 (DST) has demand of 0. Oh, but the 2am value on Nov. 6 is twice the surrounding hours, so they appear to add two hours together on one line.... (Need to remember this as something other places might do)

In [ ]:
# Download the 2012 EIA-861 data to a temp folder
url = 'https://www.eia.gov/electricity/data/eia861/archive/zip/f8612012.zip'
save_folder = cwd / "EIA861"
save_folder.mkdir(parents=True, exist_ok=True)
urllib.request.urlretrieve(url, save_folder / 'f8612012.zip')
### Unzip it
data_path = save_folder / "f8612012"
with zipfile.ZipFile(save_folder / 'f8612012.zip', 'r') as zfile:
    zfile.extractall(data_path)

In [ ]:
ferc_eia_map = pd.read_csv(cwd / "FERC" / "form714-database" / "Respondent IDs.csv", index_col=0)

# Only keep the respondents that are in the years of data we're looking at
ferc_eia_map = ferc_eia_map.loc[respondents, :]
ferc_eia_map.head()

In [ ]:
# 307 is PacifiCorp - Part II Sch 2 (East & West combined), which looks to be
# EIA utility number 14354
summary_df["eia_code"] = summary_df["name"].map(ferc_eia_map["eia_code"])
summary_df.query("~(eia_code > 0)")

In [ ]:
eia8612012_territory = pd.read_excel(cwd / "EIA861" / "f8612012" / "service_territory_2012.xls")
eia8612012_territory.head()

In [ ]:
# Document which ferc respondents can match with a utility. These utilities have
# a list of all the counties they are active in.
ferc_eia_map["utility_match"] = False
ferc_eia_map.loc[
    ferc_eia_map.eia_code.isin(eia8612012_territory['Utility Number']),
    "utility_match"
] = True

In [ ]:
eia8612012_bas = pd.read_excel(cwd / "EIA861" / "f8612012" / "balancing_authority_2012.xls")
eia8612012_bas.head()

In [ ]:
ferc_eia_map["ba_match"] = False
ferc_eia_map.loc[
    ferc_eia_map.eia_code.isin(eia8612012_bas['BA Code']),
    "ba_match"
] = True

## Examine which entities we can match to BAs and utilities

It looks like some of the respondents are only BAs, some are only utilities, and some are both. Will need to figure out if any of the utilities that are not a BA are within a BA territory - are there cases where demand from one respondent is a subset of another respondent?

In [ ]:
ferc_eia_map.query("ba_match==True")

In [ ]:
ferc_eia_map.query("utility_match==False & ba_match==False")

In [ ]:
# Some entities are both a BA and a utility
ferc_eia_map.query("utility_match==True & ba_match==True")

In [ ]:
ferc_eia_map.query("utility_match==True & ba_match==False")